In [1]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import *

from quantfreedom.simulate import run_df_backtest, or_backtest
from quantfreedom.strategies.strategy import Strategy
from quantfreedom.plotting.plotting_base import plot_or_results


mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [2]:
candles = mufex_main.get_candles(
    symbol="BTCUSDT", timeframe="5m", candles_to_dl=550)

In [3]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
# backtest_settings = BacktestSettings(
#     array_size=200,
#     gains_pct_filter=-90,
#     total_trade_filter=10,
#     upside_filter=-np.inf,
# )
backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([9, 12]),
    max_trades=np.array([5]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2, 5]),
    sl_based_on_add_pct=np.array([0.01]),
    sl_based_on_lookback=np.array([20]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5]),
    trail_sl_when_pct=np.array([1]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    logger_bool=True,
    long_or_short="long",
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type=TakeProfitFeeType.Limit,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)
strategy = Strategy(
    rsi_is_below=np.array([60, 70]),
    rsi_length=np.array([14]),
)

In [12]:
run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    static_os=static_os,
    strategy=strategy,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 2
Total order settings to test: 4
Total combinations of settings to test: 8
Total candles: 550
Total candles to test: 4,400


,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
0,0,0,28.0,11,17,6.796,39.286,0.438,418.071,67.965,1067.965
1,0,1,30.0,11,19,18.467,36.667,0.605,477.140,184.670,1184.670
2,0,2,28.0,11,17,6.796,39.286,0.438,418.071,67.965,1067.965
3,0,3,30.0,11,19,18.467,36.667,0.605,477.140,184.670,1184.670
4,1,0,28.0,11,17,7.715,39.286,0.450,420.504,77.150,1077.150
5,1,1,30.0,11,19,18.559,36.667,0.601,475.727,185.593,1185.593
6,1,2,28.0,11,17,7.715,39.286,0.450,420.504,77.150,1077.150
7,1,3,30.0,11,19,18.559,36.667,0.601,475.727,185.593,1185.593


In [10]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    static_os=static_os,
    strategy=strategy,
    dos_index=2,
    ind_set_index=0,
    plot_results=True,
)

In [11]:
order_records_df

,ind_set_idx,or_set_idx,bar_idx,timestamp,datetime,order_status,equity,available_balance,cash_borrowed,cash_used,...,entry_size_usd,entry_price,exit_price,position_size_asset,position_size_usd,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
